In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import scipy.signal
import time
import cv2
import matplotlib.pyplot as plt

In [2]:
tf.config.list_physical_devices("GPU")

[]

In [3]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [4]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [5]:
num_actions = 3
observation_dimensions = (128, 128, 3)

In [6]:
def Model():
    inp = keras.layers.Input( shape=observation_dimensions )
    x = keras.layers.Conv2D(filters=32, kernel_size=(3,3), kernel_initializer='he_normal', 
                            padding='same', activation="relu")(inp)
#     x = keras.layers.AveragePooling2D( (2,2) )(x)
#     x = keras.layers.Conv2D(filters=16, kernel_size=(3,3), kernel_initializer='he_normal', 
#                             padding='same', activation="relu")(x)
    x = keras.layers.AveragePooling2D( (2,2) )(x)
    x = keras.layers.Conv2D(filters=8, kernel_size=(3,3), kernel_initializer='he_normal', 
                            padding='same', activation="relu")(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dense(num_actions, activation="softmax")(x)
    return keras.models.Model( inp, x )

In [7]:
model = Model()
model.compile( optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
average_pooling2d (AveragePo (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 8)         2312      
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               4194432   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256  

In [8]:
import os

In [9]:
data = pd.read_csv("action_took.csv")
print(data.shape)
data = data[ data.iloc[:,0] != "0" ]
data.shape

(2149, 2)


(2148, 2)

In [10]:
images = []
for img in data.iloc[:,0]:
    images.append(cv2.imread(img).reshape((-1,128,128,3)) )
images = np.concatenate(images, axis=0)
images.shape

(2148, 128, 128, 3)

In [11]:
data.iloc[:,1].value_counts()

-1    1113
 1     558
 0     477
Name: 1, dtype: int64

In [12]:
labels = (data.iloc[:,1]+1).values

In [13]:
target = np.zeros( (len(images), 3) )
target.shape

(2148, 3)

In [14]:
for i in range(len(labels)):
    target[i][ labels[i] ] = 1

In [15]:
target.sum(axis=0)

array([1113.,  477.,  558.])

In [16]:
model.fit( images/255, target, validation_split=0.2, batch_size=128, epochs=3 )

Epoch 1/3
14/14 [==============================] - 14s 893ms/step - loss: 1.6177 - accuracy: 0.6065 - val_loss: 0.5788 - val_accuracy: 0.8860
Epoch 2/3
14/14 [==============================] - 12s 818ms/step - loss: 0.6586 - accuracy: 0.7381 - val_loss: 0.2801 - val_accuracy: 0.9209
Epoch 3/3
14/14 [==============================] - 12s 855ms/step - loss: 0.5911 - accuracy: 0.7468 - val_loss: 0.3203 - val_accuracy: 0.9186


In [20]:
model.save("classifier.h5")

In [8]:
model = keras.models.load_model("classifier.h5")

In [9]:
import gym
import gym_carla
import carla

In [ ]:
#WetCloudyNoon, WetSunset, HardRainNoon

In [44]:
params = {
    'number_of_vehicles': 0,
    'number_of_walkers': 0,
    'display_size': 250,  # screen size of bird-eye render
    'display_height' : 512,
    'display_main': True,
    'weather': "WetSunset",
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': False,  # whether to use discrete control space
    'discrete_acc': [1.0, 0.0, 1.0],  # discrete value of accelerations
    'discrete_steer': [-1, 0, 1],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.2, 0.2],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'address':'localhost',
    'port': 2000, # connection port
    'town': 'Town02', # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 5000,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 3.0,  # threshold for out of lane
    'desired_speed': 8,  # desired speed (m/s)
    'max_ego_spawn_times': 200,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
  }

In [45]:
def read_transform(img):
    return img[76:204,76:204,:]/255
env = gym.make('carla-v0', params=params)

connecting to Carla server...
Carla server connected!


In [46]:
observation =  env.reset()
for _ in range(20): observation, _, _, _ = env.step([1,0])
done = False
while not done:
    #action = np.random.choice( [-1,0,1], p=model.predict( read_transform(observation['birdeye']).reshape( (1, 128,128,3) ))[0])
    action = np.argmax(model.predict( read_transform(observation['birdeye']).reshape( (1, 128,128,3) ))[0])-1
    observation, _, done, _ = env.step( [1,action] ) 